In [12]:
def format_index(df):
    df = df.set_index(df.index.strftime('%Y.%m.%d'))
    return df

def df_styler(df, table_name=None):
    df = format_index(df)
    df_styler_out = df.style.set_table_attributes("style='display:inline'").set_caption(table_name)
    return df_styler_out

def super_styler(dfs, table_names, format_str):
    df_style_list = [df_style.format(format_str) for df_style in 
                      [df_styler(df, tbl_name) for df, tbl_name  in zip(dfs, table_names)]]
    df_style_list = [item._repr_html_() for item in df_style_list]
    str_out = ''.join(df_style_list)
    return str_out

def title_display(title):
    if spx_asset.option_asset.trading_class == 'SPXW':
        title = title + ' - Weeklys'
    display(HTML('<h1>' + title + '</h1>'))


title_display('Trade Grid')
display_html(df_styler(trd_choice.strike_grid, 'Strike Grid')._repr_html_() + 
             df_styler(trd_choice.premium_grid, 'Premium Grid')._repr_html_() + 
             df_styler(trd_choice.prices_grid, 'Prices Grid')._repr_html_(), raw=True) 

prct_decline =  format_index(trd_choice.pct_otm_grid(last_price))
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}

title_display('Strike Moneyness')
display(prct_decline.style.format(format_dict).set_caption('Strike % OTM'))

leverage_table_names = ['Leverage: ' + str(lev) for lev in leverage]
premium_list =  [(trd_choice.premium_grid * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

title_display('Premium')
display_html(super_styler(premium_list, leverage_table_names, '${:,.0f}'), raw=True)

title_display('Premium as % of capital at risk')
premium_list_pct = [df / capital_at_risk for df in premium_list]
display_html(super_styler(premium_list_pct, leverage_table_names, '{:.2%}'), raw=True)
contract_table_list = trd_choice.option_lots(leverage, capital_at_risk)

title_display('Contract to Trade')
display_html(super_styler(contract_table_list, leverage_table_names, '{:.0f}'), raw=True)

title_display('Margin')
margin_dlr_list = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
display_html(super_styler(margin_dlr_list, leverage_table_names, '${:,.0f}'), raw=True)

title_display('Notional $ Exposure')
notional_dlr=[df * trd_choice.strike_grid * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
display_html(super_styler(notional_dlr, leverage_table_names, '${:,.0f}'), raw=True)

{'cell_type': 'markdown',
 'metadata': {},
 'source': '# Notebook estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. \n\nAlgorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, [Cost_of_Capital](https://www.hbs.edu/faculty/Publication%20Files/Cost%20of%20Capital%20for%20Alternative%20Investments_57a4f444-65fa-4f0c-b51a-116408f1dab9.pdf)\n\n - Requires an open session of either IB Gateway or TWS from Interactive Brokers'}